# Homework 2: Algorithms for DNA sequencing

In [1]:
import bm_preproc
from bm_preproc import BoyerMoore
import functions #List of functions defined by me [Refer github for details]
from functions import readf
import matplotlib.pyplot as plt
import bisect
import numpy as np
from scipy.stats import norm

In [2]:
def naive_comparison_count(pattern,text):
    occurances=[]
    count_text=0
    count_pattern=0
    for t in range(len(text)-len(pattern)+1):
        count_text+=1
        match=True
        for p in range(len(pattern)):
            count_pattern+=1
            if text[p+t]!=pattern[p]:
                match=False
                break
        if match==True:
            occurances.append(t) # record the index in t where p matches
    return list(set(occurances)),count_text,count_pattern #also return whether a match was present or not

In [3]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    count_text=0
    count_pattern=0
    while i < len(t) - len(p) + 1:
        count_text+=1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            
            count_pattern+=1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences,count_text, count_pattern

In [4]:
seq_dict={}
with open('chr1.GRCh38.excerpt.fasta','r') as f:
    for r in f.readlines():
        if r[0]=='>':
            name=r
            seq=''
        else:
            seq=seq+r.rstrip()
    seq_dict[name]=seq
seq_dict.keys()

dict_keys(['>CM000663.2_excerpt EXCERPT FROM CM000663.2 Homo sapiens chromosome 1, GRCh38 reference primary assembly\n'])

In [5]:
text=seq #base data

In [6]:
alphabet='ATGC' #for all questions

<b>Question 1:</b><br>
How many alignments does the naive exact matching algorithm try when matching the string <i>GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG</i> (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [7]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, num_alignments, num_character_comparisons = naive_comparison_count(p, text)
print(f'The total number of alignments based on naive exact matching is {num_alignments}')

The total number of alignments based on naive exact matching is 799954


<br>
<br>

<b>Question 2: </b><br>
How many character comparisons does the naive exact matching algorithm try when matching the string <i>GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG</i> (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [8]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, num_alignments, num_character_comparisons = naive_comparison_count(p, text)
print(f'The total number of alignments based on naive exact matching is {num_character_comparisons}')

The total number of alignments based on naive exact matching is 984143


<br>
<br>

<b>Question 3:</b><br>
How many alignments does Boyer-Moore try when matching the string <i>GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG</i> (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [9]:
pattern = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(pattern, alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore(pattern, p_bm, text)
print(f' The number of alignments as per Boyer-Moore is {num_alignments}')

 The number of alignments as per Boyer-Moore is 127974


<br>
<br>

<b>Question 4:</b><br>
Index-assisted approximate matching. In practicals, we built a Python class called Index implementing an ordered-list version of the k-mer index.

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index. 

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string <i>GGCGCGGTGGCTCACGCCTGTAAT</i>, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm implemented in the previous module.

<br> <b>I had created my own implementation of the Index matching algorithm and have used it here. The working does not include either the kmer_index.py or the Index class</b>

In [10]:
def create_index1(text, kmer): #Create the index of all kmers from text 't' #### This is just a copy-paste from above for easy readability
    index={}
    for i in range(len(text)-kmer+1):
        if text[i:i+kmer] in index:
            index[text[i:i+kmer]]+=[i]
        else:
            index[text[i:i+kmer]]=[i]
    return index

In [11]:
def find_all_indexes(index, pattern,kmer): #does not use bisect
    sample=pattern[:kmer]
    if sample in index.keys():
        return index[sample]

In [12]:
def verify_pattern(index, text, pattern,kmer):
    hits=find_all_indexes(index, pattern, kmer)
    print('hits:',hits)
    matches=True
    occurances=[]
    for h in hits:
        for i in range(len(pattern)-kmer+1):
            if text[h+i]!=pattern[i]:
                matches=False
                break
        if matches:
            occurances.append(h)
    return occurances

In [13]:
def get_patterns(p, mismatches):
    batch_number=mismatches+1
    batch_size=int(round(len(p)//batch_number))
    print(batch_size)
    batches=[]
    for i in range(0,len(p),batch_size):
        batches.append(p[i:i+batch_size])
    return batches

In [14]:
def compare_batches_index(batches, text, pattern, index,kmer, mismatch_th=2):
    complete_matches=[]
    batch_length=len(batches[0])
    for i,batch in enumerate(batches):
        occurance_batch=verify_pattern(index, text, batch,kmer)
        for m in occurance_batch:
            if m<i*batch_length or m-(i*batch_length)+len(pattern)>len(text):                    
                continue
            mismatches=0
            for j in range(0,i*batch_length):
                if text[m-i*batch_length+j]!=pattern[j]:
                    mismatches+=1
                    if mismatches>mismatch_th:
                        break
            for j in range(min((i+1)*batch_length,len(p)),len(p)):
                if text[m-i*batch_length+j]!=pattern[j]:
                    mismatches+=1
                    if mismatches>mismatch_th:
                        break
            if mismatches<=mismatch_th:
                complete_matches.append(m-i*batch_length)
    return complete_matches 

In [15]:
p='GGCGCGGTGGCTCACGCCTGTAAT'
index_ind=create_index1(text, 8)
pattern_batches=get_patterns(p, 2)
final_occurances=compare_batches_index(pattern_batches, text, p, index_ind,kmer=8, mismatch_th=2)
print(f'Length of unqiue final occurances:{len(set(final_occurances))}')

8
hits: [56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061]
hits: [18878, 56930, 160170, 262050, 273677, 282012, 364271, 421229, 429307, 465655, 472642, 489446, 551142, 621370, 657504, 717714, 724935]
hits: [18749, 19182, 22413, 22548, 23019, 23154, 43143, 56938, 67379, 83736, 83879, 84657, 84791, 108126, 129010, 147574, 160745, 175326, 186012, 187671, 191468, 205397, 251106, 251240, 262058, 273685, 282020, 322751, 364279, 364412, 421237, 429315, 454348, 465663, 471982, 480517, 480658, 523101, 551150, 551843, 572212, 588494, 595557, 613475, 621507, 632321, 635947, 646504, 651539, 657512, 674072, 681753, 707167, 717722, 719434, 724943, 746636, 747375, 747511, 760505]
Length of unqiue final occurances:19


<b>Question 5:</b><br>
Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of <b>GGCGCGGTGGCTCACGCCTGTAAT</b> with up to 2 substitutions in the excerpt of human chromosome 1?

  (Don't consider reverse complements.)

Hint: You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer.

<b>In the previous answer, the hits arrays for each batch are given. The answer is just the sum total of these array lengths



In [16]:
hit1=[56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061]
hit2=[18878, 56930, 160170, 262050, 273677, 282012, 364271, 421229, 429307, 465655, 472642, 489446, 551142, 621370, 657504, 717714, 724935]
hit3=[18749, 19182, 22413, 22548, 23019, 23154, 43143, 56938, 67379, 83736, 83879, 84657, 84791, 108126, 129010, 147574, 160745, 175326, 186012, 187671, 191468, 205397, 251106, 251240, 262058, 273685, 282020, 322751, 364279, 364412, 421237, 429315, 454348, 465663, 471982, 480517, 480658, 523101, 551150, 551843, 572212, 588494, 595557, 613475, 621507, 632321, 635947, 646504, 651539, 657512, 674072, 681753, 707167, 717722, 719434, 724943, 746636, 747375, 747511, 760505]
print(f'Total number of index hits: {len(hit1)+len(hit2)+len(hit3)}')

Total number of index hits: 90


<b>Question 6: </b></br>
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA (first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222, and splitting into two subsequences of every other character could be represented as 121212

The class SubseqIndex is a more general implementation of Index that additionally handles subsequences. It only considers subsequences that take every Nth character

Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

<br> <b>I had created my own implementation of the subsequence implementation algorithm and have used it here. The working does not include the SubseqIndex class</b>

In [17]:
def create_subsequence(text, leng,interval):#creates subsequences of length "leng" whose initial chars are "interval" values apart
    subsequence=[]
    for i in range(len(text)):
        str1=''
        for r in range(i,len(text),interval):
            str1+=text[r]
            if len(str1)>=leng:
                break
        if len(str1)==leng:
            subsequence.append((str1,i))
        i+=1
    return subsequence

In [18]:
def create_subsequence_index(text, leng, interval):
    index={}
    subsequences=create_subsequence(text, leng, interval)
    subsequence_key,subsequence_value=[i for i in zip(*subsequences)]
    for i in range(len(subsequence_key)):
        if subsequence_key[i] in index:
            index[subsequence_key[i]].append(subsequence_value[i])
        else:
            index[subsequence_key[i]]=[subsequence_value[i]]
    return index,subsequences  

In [19]:
def find_subsequence_index(index, pattern, leng, interval):
    occurances=[]
    subsequences_pattern=create_subsequence(pattern, leng, interval)
    subsequence_key_pattern,_=[i for i in zip(*subsequences_pattern)]
    count=0
    for sub in subsequence_key_pattern:
        if sub in index[0]:
            occurances.append(index[0][sub])
        count+=1
    return occurances,len(subsequence_key_pattern),count

In [20]:
def verify_subsequence_pattern(index, text,pattern, leng, interval,mismatch_threshold=1):
    hits, pattern_key_length, total_key_matches=find_subsequence_index(index, pattern, leng, interval)
    matches=True
    occurances=[]
    index_hits=0
    for h in hits:
        h=list(h)
        for i in h:
            index_hits+=1
            mismatch_count=0
            for j in range(interval+1):
                if text[i+j]!=pattern[j]:
                    mismatch_count+=1
                    if mismatch_count>mismatch_threshold:
                        matches=False
                        break
        if matches:
            occurances.append(h)
    return np.squeeze(occurances).tolist(),index_hits

In [21]:
indexq=create_subsequence_index(text, 8, 3)
occurances, count_hits=verify_subsequence_pattern(indexq, text,'GGCGCGGTGGCTCACGCCTGTAAT',8, 3,2)
print('Total number of hits is:',count_hits)

Total number of hits is: 79
